In [1]:
import pandas as pd
import torchvision.transforms as tt
from torch.utils.data import Dataset
import os
import numpy as np
import torchvision
from PIL import Image

import torch
from torchvision import models
from torch.utils.data import DataLoader

In [2]:
class GDataset(Dataset):
    def __init__(self, df, directory):

        self.df = df
        self.dir = directory
        self.transform = tt.Compose([
            tt.Resize((160, 160)),
            tt.RandomCrop((128, 128)),
            tt.ToTensor(),
            tt.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    
    
    def __len__(self):
        return len(self.df)

    
    def __getitem__(self, index):
        
        id_ = self.df.iloc[index]['id']
        img_name = "/".join([self.dir, id_[:3], id_+".jpg"])
        image = Image.open(img_name)
        image = self.transform(image)
        return image, torch.tensor(int(self.df.iloc[index]['class_id'])).long()
        

In [ ]:
train = pd.read_csv("./train_2.csv")
train_data = GDataset(train, "train")


batch_size = 108
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

model = models.vgg16_bn(pretrained=True)
# model = models.vgg16(pretrained=False)
model.avgpool = torch.nn.AdaptiveAvgPool2d(output_size=(3, 3))
model.classifier = torch.nn.Sequential(torch.nn.Linear(4608, 4096),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(4096, 4096),
                                       torch.nn.ReLU(),
                                       torch.nn.Linear(4096, train['class_id'].max() + 1))
# model.load_state_dict(torch.load("model21.pth"))


for param in model.features[:34].parameters():
    param.requires_grad = False
model = model.cuda()

criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters())

epochs = 50

softmax = torch.nn.Softmax(dim=1)

# train the model
for e in range(epochs):

    train_loss = 0
    accuracy = 0
    counter = 0
    for x, t in train_loader:
        counter += 1
        print(round(counter / len(train_loader), 2), end="\r")
        x, t = x.cuda(), t.cuda()
        optimizer.zero_grad()
        z = model(x)
        loss = criterion(z, t)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        y = softmax(z)
        top_p, top_class = y.topk(1, dim=1)
        accuracy += (top_class[:, 0] == t).sum().item()

    print(e, train_loss / len(train_loader), accuracy / len(train_data))
    with open("summary" + str(e) + ".txt", "w") as f:
        f.write(str(train_loss / len(train_loader)) + "\n")
        f.write(str(accuracy / len(train_data)) + "\n")
    torch.save(model.state_dict(), "model" + str(e) +".pth")

0 5.434769566903667 0.05148611867174741
1 4.235207422960515 0.16759208855017238
2 3.6947302511866935 0.24432226456178552
3 3.3570723965250213 0.29549265106151335
4 3.118411796780589 0.333373253493014
5 2.924414860230628 0.3650771184902921
6 2.770350801813939 0.38997278170930866
7 2.640750273314763 0.4124768644529124
8 2.526305966476288 0.43173652694610776
9 2.4305111642541557 0.44762837960442753
10 2.339171708190703 0.46465977136635817
11 2.2626385821911237 0.4767192886953366
12 2.19331139779203 0.49021956087824353
13 2.128381524106552 0.5005951732897841
